In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
import inspect

In [6]:
'fused' in inspect.signature(torch.optim.AdamW).parameters

True